In [194]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.functional as vF
import numpy as np
import torchvision

In [156]:
def pil_to_float(image):
    return vF.convert_image_dtype(vF.pil_to_tensor(image), torch.float32)

In [514]:
'{:05}'.format(5)

'00005'

In [459]:
class BasicModel(nn.Module):
    def __init__(self):
        super(BasicModel, self).__init__()
        self.c1 = nn.Conv2d(1, 9, (3,3), padding=(1,1))
        self.p1 = nn.MaxPool2d(2, stride=2)
        self.c2 = nn.Conv2d(9, 16, (3,3), padding=(1,1))
        self.p2 = nn.MaxPool2d(2, stride=2)
        self.l1 = nn.Linear(7*7*16, 32)
        self.l2 = nn.Linear(32, 10)
        
    def forward(self, x):
        h = F.relu(self.c1(x))
        h = self.p1(h)
        h = F.relu(self.c2(h))
        h = self.p2(h)
        h = h.view(-1, 7*7*16)
        h = F.relu(self.l1(h))
        h = F.relu(self.l2(h))
        y = F.softmax(h, dim=1)
        return y

In [460]:
MAX_EPOCH = 3
LR = 0.01

In [461]:
criterion = nn.CrossEntropyLoss().cuda()
model = BasicModel().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=0.9)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lambda epoch: 0.95**epoch)

In [462]:
for epoch in range(MAX_EPOCH):
    for x, y in dataloader_train:
#         zero buffers
        optimizer.zero_grad() 
#         forward propagation
        prediction = model(x.cuda())
        
        loss = criterion(prediction, y.cuda())
        
#         gradient calculation
        loss.backward()
#         parameter update
        optimizer.step() 
    
    print('{}, {}'.format(epoch, loss), end='\r')
    scheduler.step()

In [463]:
c = 0 
w = 0
for x, y in dataset_test:
    if model(x[None,...].cuda()).argmax()==y:
        c += 1
    else:
        w -= -1
print(c/(c+w))

0.8279


In [430]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()